In [1]:
import numpy as np
import pandas as pd

In [2]:
def country_spread(file_path_str,country_name_str):
    
    ## Load File ##
    file_dir=file_path_str
    df=pd.read_excel(file_dir)
    df=df.replace(r'^\s*$', np.nan, regex=True)
    
    ## Weight(vp) DataFrame
    Countries=list(df['E1006_NAM'].unique())
    weight_list=list(df.columns[-9:])
    weight_list_countries=['E1006_NAM']+weight_list
    weight_df=df[weight_list_countries].groupby('E1006_NAM').mean(numeric_only=False)
    ## Voters(likeip) DataFrame
    voter_list=list(df.columns[2:-9])
    voter_list_countries=['E1006_NAM']+voter_list
    voter_df=df[voter_list_countries]
    
    ## Calculate spread
    
    vp_cal=weight_df.loc[country_name_str].iloc[0:,].dropna().values
    likeip_cal=voter_df[voter_df['E1006_NAM']==country_name_str].iloc[:,1:len(vp_cal)+1].dropna(axis=0)
    like_bar=likeip_cal.dot(vp_cal*0.01)
    likeip_likeBar=(likeip_cal.T-like_bar).T
    square=likeip_likeBar.apply(np.square).round(2)
    sum_vp_dot_squre=square.dot(vp_cal*0.01)
    spread=np.sqrt(np.array(sum_vp_dot_squre,dtype=np.float64))
    likeip_cal['spread']=spread.round(2)
    Contry_spread=pd.DataFrame(likeip_cal['spread'].describe().round(2)).rename(columns={"spread":country_name_str+' spread description'})
    
    ## save data ##
    Contry_spread.to_excel(country_name_str+' Contry_spread.xlsx')
    likeip_cal.to_excel(country_name_str+' likeip_cal.xlsx')
    weight_df.to_excel(country_name_str+' weight_df.xlsx')
    
    return Contry_spread,likeip_cal,vp_cal